In [3]:
import os
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.nonparametric.api as nparam
from statsmodels.nonparametric.kernel_regression import KernelReg

In [5]:
os.chdir('/home/jlm217/var_imp_mis')
from function import *

In [3]:
df = pd.read_csv('data_sim.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
n = len(df[:300])
df=df[:300]

eps = np.random.normal(0, 0.1, len(df))

a = 2
b = 2

b0=0.5
b1 = 0.5

sim = 300

p = 0.7

In [5]:
settings=nparam.EstimatorSettings(efficient=True)

In [6]:
#y = dataset(n,df).ishigami(eps)(a,b)

#model_pf = KernelReg(endog=[y.to_numpy()],exog=[df['X1'],df['X2'],df['X3']],reg_type='lc',var_type='ccc', bw='cv_ls',defaults=settings)
#y_hat_true = model_pf.fit([df['X1'],df['X2'],df['X3']])[0]

In [7]:
def sobol_reg(df,n1,request,sim,a,b,eps):
    sobol_list = []
    df1 = df.copy()
    for x in range(sim):
        #Compute y_hat
        y = dataset(n1,df1).ishigami(eps)(a,b)
        model_pf = KernelReg(endog=[y.to_numpy()],exog=[df1['X1'],df1['X2'],df1['X3']],reg_type='lc',var_type='ccc', bw='cv_ls',defaults=settings)
        y_hat = model_pf.fit([df1['X1'],df1['X2'],df1['X3']])[0]
        #Pick-Freezed
        df_pf   = dataset(n1,df).picked(request,a,b)
        df_pf['Y'] = dataset(n1,df_pf).ishigami(eps)(a,b)
        model_pf = KernelReg(endog=[df_pf['Y'].to_numpy()],exog=[df_pf['X1'],df_pf['X2'],df_pf['X3']],reg_type='lc',var_type='ccc', bw='cv_ls',defaults=settings)
        df_pf['Y_hat'] = model_pf.fit([df_pf['X1'],df_pf['X2'],df_pf['X3']])[0]
        sobol_list.append((np.cov(y_hat,df_pf['Y_hat'],bias=True)/np.var(y_hat))[0][1])
    return sobol_list

In [8]:
#sobol_reg(df,n,['X2','X3'],10,a,b,eps,y_hat)

In [9]:
def singletons_reg(df,n1,sim,a,b,eps):
    singleton = []
    for x in [['X2','X3'],['X1','X3'],['X1','X2']]:
        singleton.append(sobol_reg(df,n1,x,sim,a,b,eps))
    return pd.DataFrame(dict(zip(['X1','X2','X3'],singleton)))




In [10]:
def singleton_reg(df,Xi,n,p,sample):
    omega = pd.DataFrame(np.random.binomial(size=n, n=1, p= p),columns=[Xi])
    omega[df.drop([Xi],axis=1).columns]=  np.ones((n,2),dtype=int)
    cc = df.where(omega==1,np.nan).dropna()
    print(len(cc))
    single_sobol_cc = singletons_reg(df,n,sim,a,b,eps)
    #fig, ax = plt.subplots()
    #ax.boxplot(single_sobol_cc)
    #ax.set_xticklabels(single_sobol_cc.columns)
    single_sobol_cc['sample'] = sample
    return single_sobol_cc

In [11]:
#singleton_reg(df,'X1',n,p,'cc')

In [7]:
for x in ['X1','X2','X3']: #0.4,0.7,
    for p in [0.4,0.7,0.9]: #0.4,0.7,
        if (p - 0.4)< 1e-14:
            print(x+ ' is missing')
            print()
            print(p)
            single_sobol = singleton_reg(df,x,n,1,'complete')
            single_sobol_cc = singleton_reg(df,x,n,p,'cc')
            cdf = pd.concat([single_sobol, single_sobol_cc],axis=0)  
        else: 
            print(x+ ' is missing')
            print()
            print(p)
            cdf = singleton_reg(df,x,n,p,'cc')

        #cdf = pd.concat([single_sobol, single_sobol_cc],axis=0)  
    
        dd = pd.melt(cdf, id_vars= ['sample'],value_vars=['X1','X2','X3'],var_name='Variable')
        dd = dd.rename(columns={'value':'First Order Effect'})
        
        dd.to_csv(f'nonparametric regression/mcar_run/{x}_mcar_{p}.csv')
 
        fig, ax = plt.subplots(1, 1,figsize=(10, 5))
    
        ax = sns.boxplot(x='Variable',y='First Order Effect',data=dd,hue='sample',palette=sns.color_palette('Blues'),showfliers = False) #"Blues" "cubehelix" flare_r'
        ax.legend_.remove()
        #ax.set(ylim=(0, 0.6))
        fig = ax.get_figure()
        adjust_box_widths(fig, 0.9)
        fig.savefig('nonparametric regression/mcar/MCAR_'+str(x)+'_'+str(p)+'.pdf',transparent=True,bbox_inches='tight')


X1 is missing

0.4


NameError: name 'singleton_reg' is not defined

In [16]:
pd.read_csv(f'nonparametric regression/mcar_run/X1_mcar_0.4.csv')

,Unnamed: 0,sample,Variable,First Order Effect
0,0,complete,X1,0.253901
1,1,complete,X1,0.321824
2,2,complete,X1,0.289795
3,3,complete,X1,0.209942
4,4,complete,X1,0.301157
...,...,...,...,...
1795,1795,cc,X3,0.126550
1796,1796,cc,X3,0.114119
1797,1797,cc,X3,0.040035
1798,1798,cc,X3,0.145757


In [17]:
for x in ['X1','X2','X3']:
    dX14 = pd.read_csv(f'nonparametric regression/mcar_run/{x}_mcar_0.4.csv')
    dX14['prob'] =  dX14['sample'].apply(lambda x: 1 if x =='complete' else 0.4)

    dX16 = pd.read_csv(f'nonparametric regression/mcar_run/{x}_mcar_0.7.csv')
    #dX16 = dX16.loc[(dX16['sample']=='cc')]
    dX16['prob'] = 0.7
    print(len(dX16))

    dX19 = pd.read_csv(f'nonparametric regression/mcar_run/{x}_m0.9.csv')
    #dX19 = dX19.loc[(dX19['sample']=='cc')]
    dX19['prob'] = 0.9
    print(len(dX19))

    dp = pd.concat([dX14,dX16,dX19],axis=0,ignore_index=False)
    
    fig, ax = plt.subplots(1, 1,figsize=(12, 8))
    
    ax = sns.boxplot(x='Variable',y='First Order Effect',data=dp,hue='prob',palette=sns.color_palette('Blues'),showfliers = False,hue_order=[1,0.9,0.7,0.4]) #"Blues" "cubehelix" flare_r'
    ax.legend_.remove()
    #ax.set(ylim=(0.05, 0.58))
    fig = ax.get_figure()
    adjust_box_widths(fig, 0.9)
    plt.xticks(fontsize = 15) 
    plt.yticks(fontsize = 15) 
    ax.set_xlabel('Variable', fontsize = 20) 
    ax.set_ylabel('First Order Effect', fontsize = 20) 
    fig.savefig(f'nonparametric statistics/{x}_MCAR_p_np.pdf',transparent=True,bbox_inches='tight')

FileNotFoundError: [Errno 2] No such file or directory: 'nonparametric regression/mcar_run/X1_0.7.csv'